<a href="https://colab.research.google.com/github/pudovikoff/Experiments_tg/blob/main/Pythia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Correllation between Pythia, ETH, BTC in daily returns.

In [ ]:
This notebook download and analyze daily price changes in Pythia, ETH and BTC prices.

In [ ]:
# Install and load required packages
install.packages(c("httr", "jsonlite", "dplyr", "xts"))
library(httr)
library(jsonlite)
library(dplyr)
library(xts)

In [ ]:
# Fetch historical data function
get_historical_data <- function(crypto_id, vs_currency = "usd", days = "180") {
  url <- paste0("https://api.coingecko.com/api/v3/coins/", crypto_id, "/market_chart")
  response <- GET(url, query = list(vs_currency = vs_currency, days = days))
  data <- fromJSON(content(response, "text"))
  return(data$prices)
}

# Convert historical price data to daily returns
get_daily_returns <- function(prices) {
  # Convert list to a data frame
  prices_df <- as.data.frame(prices)
  colnames(prices_df) <- c("timestamp", "price")
  prices_df$timestamp <- as.POSIXct(prices_df$timestamp / 1000, origin = "1970-01-01")

  # Calculate daily returns
  prices_df$daily_return <- c(NA, diff(prices_df$price) / head(prices_df$price, -1))

  # Remove NA values
  prices_df <- na.omit(prices_df)

  # Convert to xts format (time series data)
  prices_xts <- xts(prices_df$daily_return, order.by = prices_df$timestamp)

  return(prices_xts)
}

# Fetch historical data for BTC, ETH, and PYTHIA
btc_prices <- get_historical_data("bitcoin")
eth_prices <- get_historical_data("ethereum")
pythia_prices <- get_historical_data("pythia")  # Assuming Pythia is available

# Convert to daily returns
btc_returns <- get_daily_returns(btc_prices)
eth_returns <- get_daily_returns(eth_prices)
pythia_returns <- get_daily_returns(pythia_prices)

# Merge the returns data by date
combined_returns <- merge(btc_returns, eth_returns, pythia_returns, all = FALSE)
colnames(combined_returns) <- c("BTC", "ETH", "PYTHIA")

# Calculate the correlation matrix of the daily returns
correlation_matrix <- cor(combined_returns)

# Print the correlation matrix
print(correlation_matrix)


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



             BTC       ETH    PYTHIA
BTC    1.0000000 0.7580616 0.1950163
ETH    0.7580616 1.0000000 0.2016643
PYTHIA 0.1950163 0.2016643 1.0000000
